In [1]:
import pandas as pd
import random, itertools
random.seed(0)
from pygeodesy.sphericalNvector import LatLon
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

In [2]:
nm2m = 1852
m2nm = 0.000539957
ft2m = 0.3048
m2ft = 3.28084

def los(callsign0, callsign1, all_tracks, distance=False):
    track0 = all_tracks[all_tracks.CallSign==callsign0]
    track1 = all_tracks[all_tracks.CallSign==callsign1]
    location0 = [LatLon(lat, lon) for lat, lon in zip(track0.Latitude, track0.Longitude)]
    location1 = [LatLon(lat, lon) for lat, lon in zip(track1.Latitude, track1.Longitude)]
    h_distances = [p0.distanceTo(p1)*m2nm for p0, p1 in zip(location0, location1)]
    v_distances = [abs(h0-h1)*m2ft for h0, h1 in zip(track0.Height, track1.Height)]
    check = [ h_dist < 5. and v_dist < 1000. for h_dist, v_dist in zip(h_distances, v_distances)]
    if distance:
        idx = h_distances.index(min(h_distances))
        return min(h_distances), v_distances[idx]
    return any(check)

In [3]:
tracks = [pd.read_csv("data/tmp/ac_{}.csv".format(i)) for i in range(0, 20)]
len(tracks)

20

In [4]:
callsigns = ["SIA{}".format(random.randint(1,999)) for i in range(0, 20)]
ownship_cs = callsigns[0]
intruder_cs = callsigns[1]
ownship_cs, intruder_cs

('SIA865', 'SIA395')

In [5]:
# update callsigns
def update_callsign(track, callsign):
    track.CallSign = callsign
    return track

In [6]:
tracks = [update_callsign(track, callsign) for track, callsign in zip(tracks, callsigns)]
_ = [track.to_csv("data/tmp/{}.csv".format(callsign), index=False) for track, callsign in zip(tracks, callsigns)]

In [7]:
all_tracks = tracks[0].append(tracks[1:], ignore_index=True)
all_tracks = all_tracks[['CallSign', 'ExerciseTime', 'Latitude', 'Longitude', 'Height']]
all_tracks.sort_values(by=['ExerciseTime'], inplace=True)
all_tracks.reset_index(drop=True, inplace=True)
all_tracks.Latitude = all_tracks.Latitude.apply(lambda x: round(x,4))
all_tracks.Longitude = all_tracks.Longitude.apply(lambda x: round(x,4))
all_tracks.Height = all_tracks.Height.apply(lambda x: round(x,4))
all_tracks.to_csv("data/tut/tutorial_full_tracks.csv", index=False)

In [8]:
# Generate simplified tracks
t0, tmax = min(all_tracks.ExerciseTime), max(all_tracks.ExerciseTime)
simple_track = all_tracks[(all_tracks.ExerciseTime==t0) | (all_tracks.ExerciseTime==tmax)].reset_index(drop=True)
simple_track.to_csv("data/tut/tutorial_simple_tracks.csv", index=False)